In [ ]:
!pip install datasets
!pip install git+https://github.com/huggingface/transformers.git


!pip install -U pip
!pip install accelerate
# !pip install appdirs
!pip install bitsandbytes
# !pip install datasets
# !pip install fire
!pip install git+https://github.com/huggingface/peft.git
# !pip install git+https://github.com/huggingface/transformers.git
!pip install torch
!pip install sentencepiece
!pip install tensorboardX
# !pip install gradio
!pip install einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 21.0 MB/s eta 0:00:00
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-t58pckn1
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-t58pckn1
  Resolved https://github.com/huggingface/transformers.git to commit ab37b801b14d8b9c3186548e6e118aff623e6aa1
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1

In [ ]:
# from transformers import set_seed,  GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling, GPT2Model,  GPT2LMHeadModel, pipeline, Trainer, TrainingArguments
from torch import tensor, nn
import pandas as pd
from datasets import Dataset, load_dataset
import numpy as np
import torch
from transformers import AdamW, get_linear_schedule_with_warmup, pipeline, set_seed
from tqdm import tqdm
import time
import re
import random
from collections import defaultdict
set_seed(42)
from peft import LoraConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("DEVICE" , DEVICE)
MODEL_NAME = "NousResearch/Llama-2-7b-hf"
print("MODEL_NAME " , MODEL_NAME)



#Hyper para
EPOCHS = 1
batch_size = 1
b = 18



#Input from user
if(b == 12):

    n = 9
    t = (3,6)

elif(b == 18):

    n = 12
    t = (4,8)










def MODEL():

    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
    )

    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        use_safetensors=True,
        quantization_config=bnb_config,
        trust_remote_code=True,
        device_map="auto",
        num_hidden_layers = b,
    )

    return model


def get_tokenizer():

    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"

    return  tokenizer


tokenizer = get_tokenizer()


m = MODEL()


print()
print()
print()
print(m)
print()
print()
print()

print()
print()
print()
print(m.config)
print()
print()
print()


# dataset= load_dataset('glue', 'sst2')

dataset= load_dataset('glue', 'qnli')

data_train_ = dataset["train"]
data_val_ = dataset["validation"]
data_test_ = dataset["test"]


def create_data_hf(d,s = "train"):


    if(s == "train"):

        n = 10

    elif(s == "val"):

        n = 10

    else:

        n = 20

    q = np.array(d['question'])[:n]
    s = np.array(d['sentence'])[:n]
    l = np.array(d["label"])[:n]
    l_ = []

    for i in l:

        if(i == 0):

            l_.append("entailment")

        else:

            l_.append("not entailment")

    l_ = np.array(l_)

    return Dataset.from_dict({  "sentence1" : q,
                                "sentence2" : s,
                                "output" : l_
                            })








print()
print()
print()
print("Data processing-1")
print()
print()
print()


data_train = create_data_hf(data_train_)
data_val = create_data_hf(data_val_)


def generate_prompt(data_point):
    return f""" Classify from the given sentences pair as 'entailment' or 'not-entailment'
### Instruction:
entailment: it is a logical relationship in which one statement or proposition necessarily implies or follows from another statement due to the laws of logic and reasoning.
not entailment: It refers to the absence of a logical relationship between two statements, where one statement does not necessarily imply or follow from the other. In other words, there is no logical consequence or necessary connection between the two statements.
### Sentence1: {data_point["sentence1"]}
### Sentence2: {data_point["sentence2"]}
### Output: {data_point["output"]}"""


def generate_prompt_test(data_point):
    return f""" Classify from the given sentences pair as 'entailment' or 'not-entailment'
### Instruction:
entailment: it is a logical relationship in which one statement or proposition necessarily implies or follows from another statement due to the laws of logic and reasoning.
not entailment: It refers to the absence of a logical relationship between two statements, where one statement does not necessarily imply or follow from the other. In other words, there is no logical consequence or necessary connection between the two statements.
### Sentence1: {data_point["sentence1"]}
### Sentence2: {data_point["sentence2"]}
### Output:"""



data_ = []
for i,j,k in zip(data_train["sentence1"], data_train["sentence2"], data_train["output"]):

  data = {
      "sentence1" : i ,
      "sentence2" : j ,
      "output" : k
  }

  data_.append(data)



max_ = float("-inf")
for i in data_:

  max_ = max(len(generate_prompt(i)),max_)

print(max_)

CUTOFF_LEN = max_






def MODEL_P(model):


    lora_r = 16
    lora_alpha = 64
    lora_dropout = 0.1
    lora_target_modules = [
        "q_proj",
        "up_proj",
        "o_proj",
        "k_proj",
        "down_proj",
        "gate_proj",
        "v_proj",
    ]


    config = LoraConfig(
        r=lora_r,
        lora_alpha=lora_alpha,
        lora_dropout=lora_dropout,
        target_modules=lora_target_modules,
        bias="none",
        task_type="CAUSAL_LM",
    )

    model = prepare_model_for_kbit_training(model)

    my_model = get_peft_model(model, config)
    my_model.print_trainable_parameters()


    return my_model



def tokenize(prompt):
    # there's probably a way to do this with the tokenizer settings
    # but again, gotta move fast
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=CUTOFF_LEN,
        padding='max_length',
        return_tensors=None,
    )

    result["labels"] = result["input_ids"].copy()

    return result


def generate_and_tokenize_prompt(data_point):

    full_prompt = generate_prompt(data_point)

    tokenized_full_prompt = tokenize(full_prompt)

    return tokenized_full_prompt

def create_data_pt(data,batch_size = batch_size):

    data.set_format(type='torch', columns=['sentence1', 'sentence2', 'output', 'input_ids', 'attention_mask', 'labels'])
    data = torch.utils.data.DataLoader(data, batch_size=batch_size)
    return data



train_d = data_train.shuffle().map(generate_and_tokenize_prompt)

val_d = data_val.shuffle().map(generate_and_tokenize_prompt)




print()
print()
print()
print("Data processing-2")
print()
print()
print()



train_d2 =  create_data_pt(train_d)
val_d2 = create_data_pt(val_d)

def pred(s,trained_model):

  inputs = tokenizer(s, return_tensors="pt").to(DEVICE)
  outputs = trained_model.generate(**inputs, max_new_tokens=50)
  out = tokenizer.decode(outputs[0], skip_special_tokens=True)
  out = out.split("\n")[6]
  return out


def trim(s):

    s = s.replace("-", " ").replace("_", " ")

    pattern = r'(entailment|not entailment)'

    matches = re.findall(pattern, s)

    if(len(matches) >= 1):

        return matches[0]

    return ""

def pred_accuracy(d,trained_model):

    X = []
    results = []

    for i,j in zip(d["sentence1"] , d["sentence2"]):

        s = generate_prompt_test({"sentence1" : i,
                                  "sentence2" : j
                                  })

        X.append(s)


    for i,j in zip(X,d["output"]):

        p = pred(i,trained_model)
        p = trim(p)



        if(len(p) == 0):

            results.append(False)

        else:

            if(p ==j):

                results.append(True)

            else:

                results.append(False)


    return results.count(True)/len(results)





def train_epoch(my_model, data_loader, optimizer,DEVICE, scheduler):



    my_model = my_model.train()
    total_train_loss = 0
    steps = 0


    for batch in tqdm(data_loader):

        b_input_ids = batch["input_ids"].to(DEVICE)
        b_masks = batch["attention_mask"].to(DEVICE)
        b_labels = batch["labels"].to(DEVICE)

        optimizer.zero_grad()

        outputs = my_model(b_input_ids,
                          labels=b_labels,
                          attention_mask = b_masks)


        # print("outputs", outputs)

        loss = outputs[0]

        batch_loss = loss.item()

        total_train_loss += batch_loss

        steps += 1

        loss.backward()
        optimizer.step()
        scheduler.step()


    return total_train_loss/steps

def eval_model(my_model, data_loader,DEVICE):

    my_model = my_model.eval()

    total_val_loss = 0
    steps = 0

    with torch.no_grad():

        for batch in tqdm(data_loader):

            b_input_ids = batch["input_ids"].to(DEVICE)
            b_masks = batch["attention_mask"].to(DEVICE)
            b_labels = batch["labels"].to(DEVICE)

            outputs = my_model(b_input_ids,
                          labels=b_labels,
                          attention_mask = b_masks)


            loss = outputs[0]

            batch_loss = loss.item()

            total_val_loss += batch_loss

            steps += 1

    return total_val_loss/steps




def train(model,DEVICE,EPOCHS,verbose = False):


    if(verbose == True):

        print()
        print()
        print()
        print(model)
        print()
        print()
        print()

    # model = model.to(DEVICE)

    tt = 0
    d = defaultdict(list)

    optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)

    total_steps = len(train_d2) * EPOCHS

    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=0,
        num_training_steps=total_steps
    )

    # Set the loss function
    loss_fn = nn.CrossEntropyLoss().to(DEVICE)

    # history = defaultdict(list)
    best_accuracy = 0


    for epoch in range(EPOCHS):

        print()
        print(f'Epoch {epoch + 1}/{EPOCHS}')
        print('-' * 20)


        s = time.time()
        train_loss = train_epoch(
        model,
        train_d2,
        optimizer,
        DEVICE,
        scheduler
        )
        e = time.time()

        print("Time per epoch for training", (e-s))


        tt += (e-s)


        val_loss = eval_model(
        model,
        val_d2,
        DEVICE,

        )

        print()
        print()
        print()
        print()
        print(f' train_loss {train_loss:.5f}   <-> Val_loss {val_loss:.5f}')
        print()
        print()
        print()
        print()


        d["train_loss"].append(train_loss)
        d["val_loss"].append(val_loss)

    print()
    print()
    print("*"*100)
    print()
    print()
    print()
    print("Total time taken: ", tt)
    print("Average time per epoch: ", tt/EPOCHS)
    print()
    print()
    print()
    print("*"*100)
    print()
    print()

    return d





def turn_off(main_model,n):

  for param in main_model.base_model.model.model.layers[n].parameters():

    param.requires_grad = False



def turn_on(main_model,n):

  arr = [False, True, True, False, True, True, False, True, True, False, True, True, False, True, True, False, True, True, False, True, True, False, False]

  for i,param in enumerate(main_model.base_model.model.model.layers[n].parameters()):

      if(arr[i] == True):

        param.requires_grad = True



def model_unfreeze(main_model):

  for i in range(b):

    turn_on(main_model,i)



def model_freeze(main_model,s):

  for i in s:

    turn_off(main_model,i)




def gen_sample(b , n):

  return random.sample(range(b), n)



def train_epoch2(my_model, data_loader, optimizer,DEVICE, scheduler,check_points,d_n,t):



    my_model = my_model.train()
    total_train_loss = 0
    steps = 0




    for i , batch in tqdm(enumerate(data_loader)):

        if(i == check_points[0] or i == check_points[1] or i == check_points[2] ):

          if(i == check_points[0]):

            model_freeze(my_model,d_n[:t[0]])

          elif(i == check_points[1]):

            model_freeze(my_model,d_n[t[0]:t[1]])

          else:

            model_freeze(my_model,d_n[t[1]:])


        b_input_ids = batch["input_ids"].to(DEVICE)
        b_masks = batch["attention_mask"].to(DEVICE)
        b_labels = batch["labels"].to(DEVICE)

        optimizer.zero_grad()

        outputs = my_model(b_input_ids,
                          labels=b_labels,
                          attention_mask = b_masks)


        # print("outputs", outputs)

        loss = outputs[0]

        batch_loss = loss.item()

        total_train_loss += batch_loss

        steps += 1

        loss.backward()
        optimizer.step()
        scheduler.step()


    return total_train_loss/steps


def train_epoch3(my_model, data_loader, optimizer,DEVICE, scheduler,d_n):



    my_model = my_model.train()
    total_train_loss = 0
    steps = 0




    for i , batch in tqdm(enumerate(data_loader)):

        model_freeze(my_model,d_n)

        b_input_ids = batch["input_ids"].to(DEVICE)
        b_masks = batch["attention_mask"].to(DEVICE)
        b_labels = batch["labels"].to(DEVICE)

        optimizer.zero_grad()

        outputs = my_model(b_input_ids,
                          labels=b_labels,
                          attention_mask = b_masks)


        # print("outputs", outputs)

        loss = outputs[0]

        batch_loss = loss.item()

        total_train_loss += batch_loss

        steps += 1

        loss.backward()
        optimizer.step()
        scheduler.step()


    return total_train_loss/steps

def train2(model,DEVICE,batch_size,EPOCHS,t,b,n,verbose = False):

    if(len(data_train) // batch_size <= 3 ):

        print("number of steps must greater than 3")
        return

    print("*"*100)
    print()
    print()
    print("Hyper Para")
    print("t : ", t)
    print("b : ", b)
    print("n : ", n)
    print()
    print()
    print("*"*100)

    if(verbose == True):

        print()
        print()
        print()
        print(model)
        print()
        print()
        print()

    check_points = np.linspace(0, batch_size, 4,endpoint = False, dtype = int)[1:]




    # model = model.to(DEVICE)

    tt = 0
    d = defaultdict(list)

    optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)

    total_steps = len(train_d2) * EPOCHS

    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=0,
        num_training_steps=total_steps
    )

    # Set the loss function
    loss_fn = nn.CrossEntropyLoss().to(DEVICE)

    # history = defaultdict(list)
    best_accuracy = 0

    for epoch in range(EPOCHS):

        d_n = gen_sample(b,n)

        model_unfreeze(model)

        print()
        print(f'Epoch {epoch + 1}/{EPOCHS}')
        print('-' * 20)

        s = time.time()
        train_loss= train_epoch2(
        model,
        train_d2,
        optimizer,
        DEVICE,
        scheduler,
        check_points,
        d_n,
        t
        )
        e = time.time()

        print("Time per epoch for training", (e-s))
        tt += (e-s)

        val_loss = eval_model(
        model,
        val_d2,
        DEVICE,

        )


        print()
        print("*"*100)
        print()
        print()
        print(f' train_loss {train_loss:.5f}   <-> Val_loss {val_loss:.5f}')
        print()
        print()
        print("*"*100)
        print()


        d["train_loss"].append(train_loss)
        d["val_loss"].append(val_loss)

    print()
    print()
    print("*"*100)
    print()
    print()
    print()
    print("Total time taken: ", tt)
    print("Average time per epoch: ", tt/EPOCHS)
    print()
    print()
    print()
    print("*"*100)
    print()
    print()

    return d


def train3(model,DEVICE,EPOCHS,b,n,verbose = False):

    print("*"*100)
    print()
    print()
    print("Hyper Para")
    print("b : ", b)
    print("n : ", n)
    print()
    print()
    print("*"*100)


    if(verbose == True):

        print()
        print()
        print()
        print(model)
        print()
        print()
        print()



    # model = model.to(DEVICE)

    tt = 0
    d = defaultdict(list)

    optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)

    total_steps = len(train_d2) * EPOCHS

    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=0,
        num_training_steps=total_steps
    )

    # Set the loss function
    loss_fn = nn.CrossEntropyLoss().to(DEVICE)

    # history = defaultdict(list)
    best_accuracy = 0

    print(EPOCHS)

    for epoch in range(EPOCHS):

        d_n = gen_sample(b,n)


        model_unfreeze(model)

        print()
        print(f'Epoch {epoch + 1}/{EPOCHS}')
        print('-' * 20)

        s = time.time()
        train_loss = train_epoch3(
        model,
        train_d2,
        optimizer,
        DEVICE,
        scheduler,
        d_n
        )
        e = time.time()

        print("Time per epoch for training", (e-s))
        tt += (e-s)

        val_loss = eval_model(
        model,
        val_d2,
        DEVICE,

        )


        print()
        print("*"*100)
        print()
        print()
        print(f' train_loss {train_loss:.5f}   <-> Val_loss {val_loss:.5f}')
        print()
        print()
        print("*"*100)
        print()


        d["train_loss"].append(train_loss)
        d["val_loss"].append(val_loss)

    print()
    print()
    print("*"*100)
    print()
    print()
    print()
    print("Total time taken: ", tt)
    print("Average time per epoch: ", tt/EPOCHS)
    print()
    print()
    print()
    print("*"*100)
    print()
    print()

    return d



if(True):

    try:


        print("#"*100)
        print()
        print()
        print("The Actual method with: ", n , " Decoders")
        print()
        print()

        m1 = MODEL_P(m)
        history1 = train(m1,DEVICE,EPOCHS)
        d = pd.DataFrame(history1)
        val = pred_accuracy(data_val,m1)

        print()
        print("*"*100)
        print()
        print("The Actual method with: ", n , " Decoders")
        print()
        print("*"*100)
        print()

        print()
        print(d)
        print()

        print()
        print("Val accuracy: " , val)
        print()



        print("#"*100)
        print()

        del m1
        torch.cuda.empty_cache()

    except Exception as e:

        print(e)



if(False):

    try:


        print("#"*100)
        print()
        print()
        print("The Actual method with t: ", t)
        print("The Actual method with n: ", n)
        print("The Actual method with b: ", b)
        print()
        print()

        m2 = MODEL_P(m)
        history2 = train2(m2,DEVICE,batch_size,EPOCHS,t,b,n)
        d = pd.DataFrame(history2)
        val = pred_accuracy(data_val,m2)

        print()
        print("*"*100)
        print()
        print("The Actual method with t: ", t)
        print("The Actual method with n: ", n)
        print("The Actual method with b: ", b)
        print()
        print("*"*100)
        print()

        print()
        print(d)
        print()

        print()
        print("Val accuracy: " , val)
        print()



        print("#"*100)
        print()

        del m2
        torch.cuda.empty_cache()

    except Exception as e:

        print(e)


if(True):

    try:


        print("#"*100)
        print()
        print()
        print("The Actual method with n: ", n)
        print("The Actual method with b: ", b)
        print()
        print()

        m3 = MODEL_P(m)
        history3 = train3(m3 ,DEVICE,EPOCHS,b,n)
        d = pd.DataFrame(history3)
        val = pred_accuracy(data_val,m3)

        print()
        print("*"*100)
        print()
        print("The Actual method with n: ", n)
        print("The Actual method with b: ", b)
        print()
        print("*"*100)
        print()

        print()
        print(d)
        print()

        print()
        print("Val accuracy: ", val)
        print()



        print("#"*100)
        print()

        del m3
        torch.cuda.empty_cache()

    except Exception as e:

        print(e)

DEVICE cuda
MODEL_NAME  NousResearch/Llama-2-7b-hf


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of the model checkpoint at NousResearch/Llama-2-7b-hf were not used when initializing LlamaForCausalLM: ['model.layers.22.input_layernorm.weight', 'model.layers.22.self_attn.rotary_emb.inv_freq', 'model.layers.24.mlp.gate_proj.weight', 'model.layers.22.mlp.down_proj.weight', 'model.layers.21.input_layernorm.weight', 'model.layers.21.self_attn.v_proj.weight', 'model.layers.31.input_layernorm.weight', 'model.layers.29.mlp.down_proj.weight', 'model.layers.27.mlp.up_proj.weight', 'model.layers.19.mlp.gate_proj.weight', 'model.layers.30.post_attention_layernorm.weight', 'model.layers.20.post_attention_layernorm.weight', 'model.layers.20.mlp.down_proj.weight', 'model.layers.23.self_attn.q_proj.weight', 'model.layers.24.input_layernorm.weight', 'model.layers.26.self_attn.v_proj.weight', 'model.layers.18.self_attn.k_proj.weight', 'model.layers.27.self_attn.q_proj.weight', 'model.layers.20.self_attn.k_proj.weight', 'model.layers.27.post_attention_layernorm.weight', 'model.layers.23




LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
     

Generating train split:   0%|          | 0/104743 [00:00<?, ? examples/s]

...

trainable params: 22,487,040 || all params: 3,927,535,616 || trainable%: 0.5725483407048497
****************************************************************************************************


Hyper Para
b :  18
n :  12


****************************************************************************************************
1

Epoch 1/1
--------------------


10it [01:13,  7.37s/it]


Time per epoch for training 73.74346351623535


100%|██████████| 10/10 [00:26<00:00,  2.61s/it]



****************************************************************************************************


 train_loss 24.45425   <-> Val_loss 19.17653


****************************************************************************************************



****************************************************************************************************



Total time taken:  73.74346351623535
Average time per epoch:  73.74346351623535



****************************************************************************************************



****************************************************************************************************

The Actual method with n:  12
The Actual method with b:  18

****************************************************************************************************


   train_loss   val_loss
0   24.454253  19.176525


Val accuracy:  0.0

####################################################################################################



In [ ]:
del m2
del m3

In [ ]:
del m

https://medium.com/deep-learning-for-protein-design/a-comprehensive-guide-to-memory-usage-in-pytorch-b9b7c78031d3

In [ ]:



def MODEL():

    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
    )

    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        use_safetensors=True,
        quantization_config=bnb_config,
        trust_remote_code=True,
        device_map="auto",
        num_hidden_layers = b,
    )

    return model


m = MODEL()



def MODEL_P(model):


    lora_r = 16
    lora_alpha = 64
    lora_dropout = 0.1
    lora_target_modules = [
        "q_proj",
        "up_proj",
        "o_proj",
        "k_proj",
        "down_proj",
        "gate_proj",
        "v_proj",
    ]


    config = LoraConfig(
        r=lora_r,
        lora_alpha=lora_alpha,
        lora_dropout=lora_dropout,
        target_modules=lora_target_modules,
        bias="none",
        task_type="CAUSAL_LM",
    )

    model = prepare_model_for_kbit_training(model)

    my_model = get_peft_model(model, config)
    my_model.print_trainable_parameters()


    return my_model




Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of the model checkpoint at NousResearch/Llama-2-7b-hf were not used when initializing LlamaForCausalLM: ['model.layers.28.self_attn.v_proj.weight', 'model.layers.27.self_attn.rotary_emb.inv_freq', 'model.layers.21.mlp.up_proj.weight', 'model.layers.20.post_attention_layernorm.weight', 'model.layers.27.self_attn.k_proj.weight', 'model.layers.30.mlp.gate_proj.weight', 'model.layers.26.mlp.down_proj.weight', 'model.layers.19.self_attn.k_proj.weight', 'model.layers.26.self_attn.k_proj.weight', 'model.layers.25.self_attn.rotary_emb.inv_freq', 'model.layers.31.self_attn.k_proj.weight', 'model.layers.26.post_attention_layernorm.weight', 'model.layers.22.self_attn.v_proj.weight', 'model.layers.25.mlp.down_proj.weight', 'model.layers.24.self_attn.k_proj.weight', 'model.layers.31.mlp.down_proj.weight', 'model.layers.31.mlp.up_proj.weight', 'model.layers.28.mlp.gate_proj.weight', 'model.layers.27.input_layernorm.weight', 'model.layers.18.input_layernorm.weight', 'model.layers.24.inpu

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [ ]:


def train_epoch_(my_model, data_loader, optimizer,DEVICE):

    f = 10**9
    device = 0

    my_model = my_model.train()
    total_train_loss = 0
    steps = 0

    d = defaultdict(list)


    print()
    print("After model to device:", torch.cuda.memory_allocated(device)/f , " GB")
    print()
    print()

    for i,batch in tqdm(enumerate(data_loader)):

        print()
        print("#"*100)
        print()
        print("i " , i )
        print()

        optimizer.zero_grad()

        a = torch.cuda.memory_allocated(device)

        b_input_ids = batch["input_ids"].to(DEVICE)
        b_masks = batch["attention_mask"].to(DEVICE)
        b_labels = batch["labels"].to(DEVICE)

        outputs = my_model(b_input_ids,
                          labels=b_labels,
                          attention_mask = b_masks)

        b = torch.cuda.memory_allocated(device)

        val1 = torch.cuda.memory_allocated(device)/f
        val2 = (b - a)/f

        print("1 - After forward pass", val1," GB")
        print("2 - Memory consumed by forward pass", val2 ," GB")


        # print("outputs", outputs)

        loss = outputs[0]

        batch_loss = loss.item()

        total_train_loss += batch_loss

        steps += 1

        loss.backward()

        val3 = torch.cuda.memory_allocated(device)/f

        print("3 - After backward pass", val3 ," GB")
        optimizer.step()

        val4 = torch.cuda.memory_allocated(device)/f

        print("4 - After optimizer step", val4 ," GB")


        d["After forward pass"].append(val1)
        d["Memory consumed by forward pass"].append(val2)
        d["After backward pass"].append(val3)
        d["After optimizer step"].append(val4)

        print()
        print("#"*100)
        print()

        if(i == 2):

            break


    return d



def train_epoch2_(my_model, data_loader, optimizer,DEVICE,d_n,t):



    f = 10**9
    device = 0

    my_model = my_model.train()
    total_train_loss = 0
    steps = 0

    d = defaultdict(list)
    check = 0


    print()
    print("After model to device:", torch.cuda.memory_allocated(device)/f , " GB")
    print()
    print()


    for i , batch in tqdm(enumerate(data_loader)):

        if(i == 8):

            break


        print()
        print("#"*100)
        print()
        print("i " , i )
        print("Checpoint: " , check)
        print()

        optimizer.zero_grad()

        a = torch.cuda.memory_allocated(device)


        if(i == 2):

            model_freeze(my_model,d_n[:t[0]])
            check += 1

        elif(i == 4):

            model_freeze(my_model,d_n[t[0]:t[1]])
            check += 1

        elif(i == 6):

            model_freeze(my_model,d_n[t[1]:])
            check +=1


        b_input_ids = batch["input_ids"].to(DEVICE)
        b_masks = batch["attention_mask"].to(DEVICE)
        b_labels = batch["labels"].to(DEVICE)



        outputs = my_model(b_input_ids,
                          labels=b_labels,
                          attention_mask = b_masks)

        b = torch.cuda.memory_allocated(device)

        val1 = torch.cuda.memory_allocated(device)/f
        val2 = (b - a)/f

        print("1 - After forward pass", val1," GB")
        print("2 - Memory consumed by forward pass", val2 ," GB")


        # print("outputs", outputs)

        loss = outputs[0]

        batch_loss = loss.item()

        total_train_loss += batch_loss

        steps += 1

        loss.backward()

        val3 = torch.cuda.memory_allocated(device)/f

        print("3 - After backward pass", val3 ," GB")
        optimizer.step()

        val4 = torch.cuda.memory_allocated(device)/f

        print("4 - After optimizer step", val4 ," GB")


        d["After forward pass"].append(val1)
        d["Memory consumed by forward pass"].append(val2)
        d["After backward pass"].append(val3)
        d["After optimizer step"].append(val4)

        print()
        print("#"*100)
        print()



    return d


def train_epoch3_(my_model, data_loader, optimizer,DEVICE,d_n):



    my_model = my_model.train()
    total_train_loss = 0
    steps = 0
    device = 0
    f = 10**9



    d = defaultdict(list)


    print()
    print("After model to device:", torch.cuda.memory_allocated(device)/f , " GB")
    print()
    print()


    for i , batch in tqdm(enumerate(data_loader)):


        print()
        print("#"*100)
        print()
        print("i " , i )
        print()

        optimizer.zero_grad()


        a = torch.cuda.memory_allocated(device)

        model_freeze(my_model,d_n)

        b_input_ids = batch["input_ids"].to(DEVICE)
        b_masks = batch["attention_mask"].to(DEVICE)
        b_labels = batch["labels"].to(DEVICE)



        outputs = my_model(b_input_ids,
                          labels=b_labels,
                          attention_mask = b_masks)


        # print("outputs", outputs)

        b = torch.cuda.memory_allocated(device)

        val1 = torch.cuda.memory_allocated(device)/f
        val2 = (b - a)/f

        print("1 - After forward pass", val1," GB")
        print("2 - Memory consumed by forward pass", val2 ," GB")


        # print("outputs", outputs)

        loss = outputs[0]

        batch_loss = loss.item()

        total_train_loss += batch_loss

        steps += 1

        loss.backward()

        val3 = torch.cuda.memory_allocated(device)/f

        print("3 - After backward pass", val3 ," GB")
        optimizer.step()

        val4 = torch.cuda.memory_allocated(device)/f

        print("4 - After optimizer step", val4 ," GB")


        d["After forward pass"].append(val1)
        d["Memory consumed by forward pass"].append(val2)
        d["After backward pass"].append(val3)
        d["After optimizer step"].append(val4)

        print()
        print("#"*100)
        print()

        if(i == 2):

            break


    return d




def train_(model,DEVICE,EPOCHS,verbose = False):


    if(verbose == True):

        print()
        print()
        print()
        print(model)
        print()
        print()
        print()

    # model = model.to(DEVICE)


    optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)

    total_steps = len(train_d2) * EPOCHS



    # Set the loss function
    # loss_fn = nn.CrossEntropyLoss().to(DEVICE)

    # history = defaultdict(list)
    best_accuracy = 0


    for epoch in range(EPOCHS):

        print()
        print(f'Epoch {epoch + 1}/{EPOCHS}')
        print('-' * 20)


        d = train_epoch_(
        model,
        train_d2,
        optimizer,
        DEVICE,

        )





    print()
    print()
    print()
    print()
    D = pd.DataFrame(d)
    print(D)
    print()
    print()
    print()
    print()
    print()




    return d





def turn_off(main_model,n):

  for param in main_model.base_model.model.model.layers[n].parameters():

    param.requires_grad = False



def turn_on(main_model,n):

  arr = [False, True, True, False, True, True, False, True, True, False, True, True, False, True, True, False, True, True, False, True, True, False, False]

  for i,param in enumerate(main_model.base_model.model.model.layers[n].parameters()):

      if(arr[i] == True):

        param.requires_grad = True



def model_unfreeze(main_model):

  for i in range(b):

    turn_on(main_model,i)



def model_freeze(main_model,s):

  for i in s:

    turn_off(main_model,i)




def gen_sample(b , n):

  return random.sample(range(b), n)




def train2_(model,DEVICE,batch_size,EPOCHS,t,b,n,verbose = False):

    if(len(data_train) // batch_size <= 3 ):

        print("number of steps must greater than 3")
        return

    print("*"*100)
    print()
    print()
    print("Hyper Para")
    print("t : ", t)
    print("b : ", b)
    print("n : ", n)
    print()
    print()
    print("*"*100)

    if(verbose == True):

        print()
        print()
        print()
        print(model)
        print()
        print()
        print()

    check_points = np.linspace(0, batch_size, 4,endpoint = False, dtype = int)[1:]




    # model = model.to(DEVICE)

    # tt = 0
    # d = defaultdict(list)

    optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)

    # total_steps = len(train_d2) * EPOCHS

    # scheduler = get_linear_schedule_with_warmup(
    #     optimizer,
    #     num_warmup_steps=0,
    #     num_training_steps=total_steps
    # )

    # # Set the loss function
    # loss_fn = nn.CrossEntropyLoss().to(DEVICE)

    # # history = defaultdict(list)
    # best_accuracy = 0

    for epoch in range(EPOCHS):

        d_n = gen_sample(b,n)

        model_unfreeze(model)

        print()
        print(f'Epoch {epoch + 1}/{EPOCHS}')
        print('-' * 20)


        d = train_epoch2_(
        model,
        train_d2,
        optimizer,
        DEVICE,
        d_n,
        t
        )


    print()
    print()
    print()
    print()
    D = pd.DataFrame(d)
    print(D)
    print()
    print()
    print()
    print()
    print()




    return d



def train3_(model,DEVICE,EPOCHS,b,n,verbose = False):

    print("*"*100)
    print()
    print()
    print("Hyper Para")
    print("b : ", b)
    print("n : ", n)
    print()
    print()
    print("*"*100)


    if(verbose == True):

        print()
        print()
        print()
        print(model)
        print()
        print()
        print()



    # model = model.to(DEVICE)



    optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)

    total_steps = len(train_d2) * EPOCHS



    # Set the loss function
    # loss_fn = nn.CrossEntropyLoss().to(DEVICE)

    # history = defaultdict(list)


    for epoch in range(EPOCHS):

        d_n = gen_sample(b,n)


        model_unfreeze(model)

        print()
        print(f'Epoch {epoch + 1}/{EPOCHS}')
        print('-' * 20)


        d = train_epoch3_(
        model,
        train_d2,
        optimizer,
        DEVICE,
        d_n
        )


    print()
    print()
    print()
    print()
    D = pd.DataFrame(d)
    print(D)
    print()
    print()
    print()
    print()
    print()




    return d







In [ ]:

if(True):

    try:

        torch.cuda.empty_cache()
        print("#"*100)
        print()
        print()
        print("The Actual method with: ", n , " Decoders")
        print()
        print()

        m1 = MODEL_P(m)
        history1 = train_(m1,DEVICE,EPOCHS)
        print()
        print()
        print()
        print()

        del m1
        torch.cuda.empty_cache()

    except Exception as e:

        print(e)


####################################################################################################


The Actual method with:  12  Decoders




/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


trainable params: 22,487,040 || all params: 3,927,535,616 || trainable%: 0.5725483407048497

Epoch 1/1
--------------------

After model to device: 3.261287424  GB




0it [00:00, ?it/s]


####################################################################################################

i  0

1 - After forward pass 3.642615808  GB
2 - Memory consumed by forward pass 0.381328384  GB
3 - After backward pass 3.455591424  GB


...

4 - After optimizer step 3.635487744  GB

####################################################################################################


####################################################################################################

i  1

1 - After forward pass 3.843784704  GB
2 - Memory consumed by forward pass 0.29824512  GB
3 - After backward pass 3.635487744  GB


2it [00:11,  5.94s/it]

4 - After optimizer step 3.635487744  GB

####################################################################################################


####################################################################################################

i  2

1 - After forward pass 3.843784704  GB
2 - Memory consumed by forward pass 0.29824512  GB
3 - After backward pass 3.635487744  GB


2it [00:18,  9.01s/it]

4 - After optimizer step 3.635487744  GB

####################################################################################################





   After forward pass  Memory consumed by forward pass  After backward pass  \
0            3.642616                         0.381328             3.455591   
1            3.843785                         0.298245             3.635488   
2            3.843785                         0.298245             3.635488   

   After optimizer step  
0              3.635488  
1              3.635488  
2              3.635488  











In [ ]:

if(True):

    try:

        torch.cuda.empty_cache()

        print("#"*100)
        print()
        print()
        print("The Actual method with t: ", t)
        print("The Actual method with n: ", n)
        print("The Actual method with b: ", b)
        print()
        print()

        m2 = MODEL_P(m)
        history2 = train2_(m2,DEVICE,batch_size,EPOCHS,t,b,n)
        print()
        print()
        print()
        print()

        del m2
        torch.cuda.empty_cache()

    except Exception as e:

        print(e)




####################################################################################################


The Actual method with t:  (4, 8)
The Actual method with n:  12
The Actual method with b:  18




/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


trainable params: 22,487,040 || all params: 3,927,535,616 || trainable%: 0.5725483407048497
****************************************************************************************************


Hyper Para
t :  (4, 8)
b :  18
n :  12


****************************************************************************************************

Epoch 1/1
--------------------

After model to device: 3.261287424  GB




0it [00:00, ?it/s]


####################################################################################################

i  0
Checpoint:  0

1 - After forward pass 3.642615808  GB
2 - Memory consumed by forward pass 0.381328384  GB
3 - After backward pass 3.455591424  GB


...

4 - After optimizer step 3.595510784  GB

####################################################################################################


####################################################################################################

i  6
Checpoint:  2

1 - After forward pass 3.843784704  GB
2 - Memory consumed by forward pass 0.29824512  GB
3 - After backward pass 3.575522304  GB


7it [00:41,  5.76s/it]

4 - After optimizer step 3.575522304  GB

####################################################################################################


####################################################################################################

i  7
Checpoint:  3

1 - After forward pass 3.843784704  GB
2 - Memory consumed by forward pass 0.29824512  GB
3 - After backward pass 3.575522304  GB


8it [00:47,  5.88s/it]

4 - After optimizer step 3.575522304  GB

####################################################################################################





   After forward pass  Memory consumed by forward pass  After backward pass  \
0            3.642616                         0.381328             3.455591   
1            3.843785                         0.298245             3.635488   
2            3.843785                         0.298245             3.615499   
3            3.843785                         0.298245             3.615499   
4            3.843785                         0.298245             3.595511   
5            3.843785                         0.298245             3.595511   
6            3.843785                         0.298245             3.575522   
7            3.843785                         0.298245             3.575522   

   After optimizer step  
0              3.635488  
1              3.635488  
2              3.615499  
3              3.615499  
4         

In [ ]:
if(True):

    try:

        torch.cuda.empty_cache()

        print("#"*100)
        print()
        print()
        print("The Actual method with n: ", n)
        print("The Actual method with b: ", b)
        print()
        print()

        m3 = MODEL_P(m)
        history3 = train3_(m3 ,DEVICE,EPOCHS,b,n)
        print()
        print()
        print()
        print()

        del m3
        torch.cuda.empty_cache()

    except Exception as e:

        print(e)

####################################################################################################


The Actual method with n:  12
The Actual method with b:  18




/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


trainable params: 22,487,040 || all params: 3,927,535,616 || trainable%: 0.5725483407048497
****************************************************************************************************


Hyper Para
b :  18
n :  12


****************************************************************************************************

Epoch 1/1
--------------------

After model to device: 3.261287424  GB




0it [00:00, ?it/s]


####################################################################################################

i  0

1 - After forward pass 3.642615808  GB
2 - Memory consumed by forward pass 0.381328384  GB
3 - After backward pass 3.395625984  GB


...

4 - After optimizer step 3.455591424  GB

####################################################################################################


####################################################################################################

i  1

1 - After forward pass 3.723853824  GB
2 - Memory consumed by forward pass 0.29824512  GB
3 - After backward pass 3.455591424  GB


2it [00:10,  5.30s/it]

4 - After optimizer step 3.455591424  GB

####################################################################################################


####################################################################################################

i  2

1 - After forward pass 3.723853824  GB
2 - Memory consumed by forward pass 0.29824512  GB
3 - After backward pass 3.455591424  GB


2it [00:16,  8.10s/it]

4 - After optimizer step 3.455591424  GB

####################################################################################################





   After forward pass  Memory consumed by forward pass  After backward pass  \
0            3.642616                         0.381328             3.395626   
1            3.723854                         0.298245             3.455591   
2            3.723854                         0.298245             3.455591   

   After optimizer step  
0              3.455591  
1              3.455591  
2              3.455591  











TypeError: ignored